In [1]:
import pandas as pd 
import numpy as np

In [2]:
df_validData = pd.read_csv("/Volumes/TwoTeras/0_Experiment_1/Eye_Tracking/Pre_processed/Data_Sets/Experiment_1_OpenEyes.cvs")

In [3]:
# Filter valid fixations
df_fix = df_validData[df_validData['isFix'].notna()].copy()

# Remove invalid samples (e.g., z-component must be > 0.7)
df_fix = df_fix[df_fix['eyeDirectionCombinedLocal.z'] > 0.7]

# Clean up: drop Unnamed columns if present
unnamed_cols = [col for col in df_fix.columns if col.startswith('Unnamed')]
df_fix.drop(columns=unnamed_cols, inplace=True)

# Prepare gaze direction vectors
LocalX = df_fix['eyeDirectionCombinedLocal.x'].to_numpy()
LocalY = df_fix['eyeDirectionCombinedLocal.y'].to_numpy()
LocalZ = df_fix['eyeDirectionCombinedLocal.z'].to_numpy()

# Stack into N x 3 matrix of local gaze directions
LocalGazeArrays = np.column_stack((LocalX, LocalY, LocalZ))

# Define nose vector
NoseVector = np.array([0, 0, 1])


In [4]:
# -- Assume df_fix and LocalGazeArrays already prepared as above --

# Create sidewards and upwards projection vectors
SidewardsX = np.column_stack((df_fix['eyeDirectionCombinedLocal.x'], df_fix['eyeDirectionCombinedLocal.z']))
UpwardsY = np.column_stack((df_fix['eyeDirectionCombinedLocal.y'], df_fix['eyeDirectionCombinedLocal.z']))

# Function: angle between two vectors in 2D plane
def angle_between(v1, v2):
    ang1 = np.arctan2(v1[1], v1[0])
    ang2 = np.arctan2(v2[1], v2[0])
    return np.rad2deg((ang1 - ang2) % (2 * np.pi))

# --- Sidewards angle changes (X-Z plane) ---
side_angle_diffs = [angle_between(SidewardsX[i-1], SidewardsX[i]) for i in range(1, len(SidewardsX))]
side_angle_diffs.append(0)  # same length
# Wrap angles: if >180, subtract 360
df_fix['SidewardsAngle_X'] = [a - 360 if a > 180 else a for a in side_angle_diffs]

# --- Upwards angle changes (Y-Z plane) ---
up_angle_diffs = [angle_between(UpwardsY[i-1], UpwardsY[i]) for i in range(1, len(UpwardsY))]
up_angle_diffs.append(0)  # same length
df_fix['UpwardsAngle_Y'] = [a - 360 if a > 180 else a for a in up_angle_diffs]


In [5]:
# --- Parameters ---
angle_threshold = 5  # degrees threshold for accumulated flip

# --- Storage lists ---
angles_list = []
idxs = []
gidxs = []
subjects = []
sessions = []
subsections = []

# --- Grouping and accumulation logic ---
for subject in df_fix['SubjectID'].unique():
    df_sub = df_fix[df_fix['SubjectID'] == subject]
    for session in df_sub['Session'].unique():
        df_sess = df_sub[df_sub['Session'] == session]
        for subsection in df_sess['SessionSubsection'].unique():
            df_part = df_sess[df_sess['SessionSubsection'] == subsection].reset_index()
            sum_angles = 0
            for i in range(len(df_part) - 1):
                sum_angles += df_part['UpwardsAngle_Y'][i]
                if abs(sum_angles) >= angle_threshold:
                    angles_list.append(sum_angles)
                    gidxs.append(df_part['index'][i])   # global index in original df_fix
                    idxs.append(i)                       # local index in df_part
                    subjects.append(subject)
                    sessions.append(session)
                    subsections.append(subsection)
                    sum_angles = 0  # reset after threshold is reached

# --- Create summary DataFrame ---
df_5gedFlip = pd.DataFrame({
    'UpwardsAngle_Y': angles_list,
    'idxs': idxs,
    'gidxs': gidxs,
    'SubjectID': subjects,
    'Session': sessions,
    'SessionSubsection': subsections
})

In [6]:
# --- Initialize filtered storage ---
angles_list = []
idxs = []
gidxs = []
subjects = []
sessions = []
subsections = []

# --- Remove consecutive flips in the same direction ---
for i in range(len(df_5gedFlip) - 1):
    cur = df_5gedFlip['UpwardsAngle_Y'].iloc[i]
    nex = df_5gedFlip['UpwardsAngle_Y'].iloc[i + 1]
    if np.sign(cur) != np.sign(nex):
        angles_list.append(cur)
        gidxs.append(df_5gedFlip['gidxs'].iloc[i])
        idxs.append(df_5gedFlip['idxs'].iloc[i])
        subjects.append(df_5gedFlip['SubjectID'].iloc[i])
        sessions.append(df_5gedFlip['Session'].iloc[i])
        subsections.append(df_5gedFlip['SessionSubsection'].iloc[i])

# --- Overwrite df_5gedFlip with cleaned entries ---
df_5gedFlip = pd.DataFrame({
    'UpwardsAngle_Y': angles_list,
    'idxs': idxs,
    'gidxs': gidxs,
    'SubjectID': subjects,
    'Session': sessions,
    'SessionSubsection': subsections
})


In [7]:
df_5gedFlip

UpwardsAngle_Y   idxs   gidxs  SubjectID  Session  SessionSubsection
0            -5.949991     47      48        365        1                  1
1             9.733817     65      67        365        1                  1
2            -6.197875     96     104        365        1                  1
3             5.372207    113     126        365        1                  1
4            -5.313497    164     191        365        1                  1
...                ...    ...     ...        ...      ...                ...
132674       -9.693842  24059  112840       9586        5                  3
132675       11.529712  24117  112930       9586        5                  3
132676      -12.659913  24162  112985       9586        5                  3
132677        9.078150  24180  113020       9586        5                  3
132678       -5.042567  24831  114070       9586        5                  3

[132679 rows x 6 columns]

In [9]:
# --- Count number of Y-axis flips per SubjectID ---
nr_flips = []
subject_ids = []

for subject in df_5gedFlip['SubjectID'].unique():
    df_sub = df_5gedFlip[df_5gedFlip['SubjectID'] == subject]
    nr_flips.append(len(df_sub) - 1)  # minus 1 to match original logic
    subject_ids.append(subject)

# --- Create summary DataFrame ---
OverallFlips = pd.DataFrame({
    'SubjectID': subject_ids,
    'nr_flips_y': nr_flips
})
OverallFlips

SubjectID  nr_flips_y
0         365        3659
1        1754        5830
2        2258        5060
3        2361        3989
4        2693        5762
5        3246        2849
6        3310        4733
7        3572        4194
8        3976        3393
9        4176        4657
10       4796        3510
11       4917        3518
12       5238        3384
13       5531        4389
14       5741        5026
15       6642        6598
16       7093        4156
17       7264        4259
18       7412        3629
19       7842        3737
20       8007        4503
21       8469        4125
22       8673        6783
23       8695        3793
24       9472        6916
25       9502        3812
26       9601        5220
27        479        4109
28       4597        3646
29       9586        3410

## X-axis (left-right on screen)

In [10]:
# --- Parameters ---
angle_threshold = 5  # degrees threshold for accumulated flip

# --- Storage lists ---
angles_list = []
idxs = []
gidxs = []
subjects = []
sessions = []
subsections = []

# --- Grouping and accumulation logic for SidewardsAngle_X ---
for subject in df_fix['SubjectID'].unique():
    df_sub = df_fix[df_fix['SubjectID'] == subject]
    for session in df_sub['Session'].unique():
        df_sess = df_sub[df_sub['Session'] == session]
        for subsection in df_sess['SessionSubsection'].unique():
            df_part = df_sess[df_sess['SessionSubsection'] == subsection].reset_index()
            sum_angles = 0
            for i in range(len(df_part) - 1):
                sum_angles += df_part['SidewardsAngle_X'][i]
                if abs(sum_angles) >= angle_threshold:
                    angles_list.append(sum_angles)
                    gidxs.append(df_part['index'][i])   # global index
                    idxs.append(i)                       # local index
                    subjects.append(subject)
                    sessions.append(session)
                    subsections.append(subsection)
                    sum_angles = 0  # reset

# --- Create DataFrame with sideward flips ---
df_5gedFlip_side = pd.DataFrame({
    'SidewardsAngle_X': angles_list,
    'idxs': idxs,
    'gidxs': gidxs,
    'SubjectID': subjects,
    'Session': sessions,
    'SessionSubsection': subsections
})


In [11]:
# --- Initialize filtered storage ---
angles_list = []
idxs = []
gidxs = []
subjects = []
sessions = []
subsections = []

# --- Filter consecutive flips in same direction ---
for i in range(len(df_5gedFlip_side) - 1):
    cur = df_5gedFlip_side['SidewardsAngle_X'].iloc[i]
    nex = df_5gedFlip_side['SidewardsAngle_X'].iloc[i + 1]
    if np.sign(cur) != np.sign(nex):
        angles_list.append(cur)
        gidxs.append(df_5gedFlip_side['gidxs'].iloc[i])
        idxs.append(df_5gedFlip_side['idxs'].iloc[i])
        subjects.append(df_5gedFlip_side['SubjectID'].iloc[i])
        sessions.append(df_5gedFlip_side['Session'].iloc[i])
        subsections.append(df_5gedFlip_side['SessionSubsection'].iloc[i])

# --- Create cleaned DataFrame ---
df_5gedFlip_side = pd.DataFrame({
    'SidewardsAngle_X': angles_list,
    'idxs': idxs,
    'gidxs': gidxs,
    'SubjectID': subjects,
    'Session': sessions,
    'SessionSubsection': subsections
})


In [12]:
# --- Count number of X-axis (sidewards) flips per SubjectID ---
nr_flips = []
subject_ids = []

for subject in df_5gedFlip_side['SubjectID'].unique():
    df_sub = df_5gedFlip_side[df_5gedFlip_side['SubjectID'] == subject]
    nr_flips.append(len(df_sub))  # no -1 here, we keep all flips
    subject_ids.append(subject)

# --- Add to OverallFlips (merge on SubjectID) ---
df_flips_x = pd.DataFrame({
    'SubjectID': subject_ids,
    'nr_flips_x': nr_flips
})

# Merge with existing upward flip counts
OverallFlips = pd.merge(OverallFlips, df_flips_x, on='SubjectID', how='outer')
OverallFlips


SubjectID  nr_flips_y  nr_flips_x
0         365        3659       13014
1         479        4109       10652
2        1754        5830       19870
3        2258        5060       17247
4        2361        3989       12048
5        2693        5762       16638
6        3246        2849        9259
7        3310        4733        9144
8        3572        4194       12057
9        3976        3393       10693
10       4176        4657       12817
11       4597        3646       15983
12       4796        3510       10617
13       4917        3518       13087
14       5238        3384        8939
15       5531        4389       13505
16       5741        5026       10737
17       6642        6598       14965
18       7093        4156       12076
19       7264        4259       12167
20       7412        3629       14427
21       7842        3737        8238
22       8007        4503       10394
23       8469        4125       10102
24       8673        6783       13445
25       8695        3793       11921
26       9472        6916       12754
27       9502        3812       11799
28       9586        3410        8397
29       9601        5220       17081

In [13]:
OverallFlips.to_csv("/Volumes/TwoTeras/PCA/Data_Sets/Experiment_1/Flips_X_Y_LargerThan5Deg.csv", index=False)
